In [1]:
import pandas as pd

# EXPERIMENT DESIGN

## SOURCES INGESTION

# EXPERIMENT RESULTS

## SOURCES INGESTION

## DATA CLEANING

## METRICS DEFINITION

## DATA ANALYSIS

### NUMBER OF VARIANTS

In [ ]:
# if number of variantes > 2

### SAMPLE SIZES

In [ ]:
# check if the sample size is large enough
# check if the variant sizes ar equal or not

### SAMPLE DISTRIBUTION

In [ ]:
# check if the distribution is normal or not

### SAMPLE VARIANCES

In [ ]:
# check if the variances are similar between variantes

# if are not equall, use_mann_whitney_u_test, use_bootstraping

## STATISTICAL TESTING

In [17]:
# two variants
use_mann_whitney_u_test = False # continuous, not normal, works with unbalanced samples
use_welchs_t_test = False # continuous, normal, different variances, works with unbalanced samples
use_t_test = False # continuous, normal, equal variances, not recommended if unbalanced
## for proportions, only size matters due to central limit theorem
use_z_test = False # continuous data, large sample size, known population variance, not recommended if unbalanced

use_fisher_exact_test = False # proportions, small sample size, works with unbalanced samples
use_two_proportion_z_test = False # proportions, large sample size, works with unbalanced samples, used alone or post Pearson Chi-square with Bonferroni for multiple variants

# multiple variants
use_anova_test = False # continuous, normal, not recommended if unbalanced
use_welch_anova_test = False # continuous, normal, if unbalanced
use_pearson_chi_square_test = False # proportions, works with unbalanced samples but needs correction
use_kruskal_wallis_test = False # continuous, not normal, works with unbalanced samples

# post multiple variants
use_tukey_hsd_test = False # post anova, continuous, not recommended if unbalanced
use_games_howell_test = False # post anova, continuous, for unbalanced samples
use_dunn_test = False # post kruskal wallis, continuous, needs bonferroni, works with unbalanced samples

# post multiple variants correction
use_bonferroni_correction = False # if more than 2 variants and tukey pr games howell is not used

# advance techniques
use_balance_resampling = False # for unbalanced sample sizes, equalizing groups
use_bootstraping = False # for small sample sizes or unbalanced groups, estimating confidence intervals
use_bayesian_test = False # for probabilistic interpretation, alternative to p-values
use_permutation_test = False # for distribution-free significance testing, alternative to t-tests or z-tests


### ADDITIONAL TECHNIQUES

#### BALANCE RESAMPLING

#### BOOTSTRAPING

#### BAYESIAN TEST

#### PERMUTATION TEST